## Naive Baye's classifier

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mlt
import csv
import random
import operator
import math
from sklearn.metrics import precision_recall_fscore_support as score

### Splitting the dataset in 80%-20%

In [72]:
train_df=[]
validation_df=[]
split = 0.2
df=pd.read_csv('loan_data.csv',header=None)
df=df[df.columns[[1,2,3,4,5,6,7,8,10,11,12,13,9]]]
dataset=list(df.values)
#df.head()

# differentiate categorical and numerical data to calculate probability using different method
categorical_features=[0,3,6,9,10,11,12]
nuberical_features=[1,2,4,5,6,7,8]
print('age:',dataset[0][categorical_features[0]])

[1.0000e+00 1.1000e+00 4.9000e+01 9.1107e+04 4.0000e+00 1.6000e+00
 1.2000e+00 0.0000e+00 1.3000e+00 2.0000e-01 3.0000e-01 4.0000e-01
 1.0000e+00]
age: 1.0


### Splitting dataset

In [73]:
def divide(df,validation_data_size):
    indices = df.index.tolist()
    validation_data_size=round(validation_data_size*len(df))
    validation_indices = random.sample(population=indices, k=validation_data_size)
    train_data = df.drop(validation_indices)
    validation_data = df.loc[validation_indices]
    return train_data,validation_data

In [75]:
random.seed(0)
train_df,validation_df = divide(df,validation_data_size=0.2)
train_data =train_df.values
validation_data=validation_df.values
for d in validation_data:
    if d[0]>=0 and d[0]<=35:
        d[0]=1
    elif d[0]>=36 and d[0]<=50:
        d[0]=2;
    else: 
        d[0]=3
for d in train_data:
    if d[0]>=0 and d[0]<=35:
        d[0]=1
    elif d[0]>=36 and d[0]<=50:
        d[0]=2;
    else: 
        d[0]=3
print(train_data[0])

[3.0000e+00 3.5000e+01 4.0000e+01 9.4536e+04 4.0000e+00 4.0000e-01
 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00]


### Functions for calculate mean and standard deviation

In [76]:
def get_mean(attribute_values):
    return sum(attribute_values)/float(len(attribute_values))

def get_standard_deviation(attribute_values):
    m=get_mean(attribute_values)
    v=sum([pow(xd-m,2) for xd in attribute_values])/float(len(attribute_values))
    return math.sqrt(v)

### Calculate Gaussian probability

In [107]:
def get_regular_prob(index,class_label,target_class,train_data):
    label_col=train_data[:,index]
    #print(label_col)
    target_label_col=train_data[:,-1]
    #print(target_label_col)
    label_count=0
    target_label_count=0
    target_label_equal=0
    for e in target_label_col:
        if e==target_class:
            target_label_count+=1
    for i in range(len(label_col)):
        if label_col[i]==class_label and target_label_col[i]==target_class:
            target_label_equal+=1
    if target_label_equal==0:
        target_label_equal+=1
    return (float(target_label_equal/target_label_count))
    
def get_gaussian_prob(t,mean,stdev):
    exponent = math.exp(-(math.pow(t-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

print(train_data)
print(get_regular_prob(0,1.0,1.0,train_data))

[[ 3. 35. 40. ...  0.  0.  0.]
 [ 2. 14. 80. ...  1.  0.  0.]
 [ 3. 32. 19. ...  0.  0.  0.]
 ...
 [ 3. 26. 45. ...  0.  0.  0.]
 [ 2. 12. 84. ...  1.  0.  0.]
 [ 3. 25. 70. ...  1.  0.  0.]]
0.2761627906976744


### Get all classes from target variable

In [108]:
def get_target_classes(dataset):
    target_classes={}
    for entry in dataset:
        if (entry[-1] not in target_classes):
            #add new target class in dictionary
            target_classes[entry[-1]] = []
        target_classes[entry[-1]].append(entry)
    return target_classes

### Group together attributes and calculate mean and standard deviation of train dataset

In [109]:

def get_mean_stdev_dataset(train_data):
    attribute_list=zip(*train_data)
    mean_stdev_dataset = [(get_mean(attribute),get_standard_deviation(attribute)) for attribute in attribute_list]
    del mean_stdev_dataset[-1]
    return mean_stdev_dataset
#for i in attribute_list:
 #   print (i)

### Get mean and standard deviation of eacj target value and their instances it includes

In [110]:
def get_mean_stdev_by_class(train_data):
    target_classes = get_target_classes(train_data)
    mean_stdev_class = {}
    for class_value, entries in target_classes.items():
        mean_stdev_class[class_value]=get_mean_stdev_dataset(entries)
    return mean_stdev_class
get_mean_stdev_mean(train_data)

NameError: name 'get_mean_stdev_mean' is not defined

In [111]:
def get_classes_prob(mean_stdev_class,train_data,test_entry):
    prob_dict = {}
    for class_value,entries in mean_stdev_class.items():
        prob_dict[class_value]=1
        for i in range(len(entries)):
            if i not in categorical_features:
                mean, stdev = entries[i]
                t=test_entry[i]
                prob_dict[class_value]*=get_gaussian_prob(t, mean, stdev)
            else:
                prob_dict[class_value]*=get_regular_prob(i,test_entry[i],class_value,train_data)
    return prob_dict

### Prediction algorithm

In [112]:
def predict(mean_stdev_class,train_data, test_entry):
    probabilities = get_classes_prob(mean_stdev_class,train_data,test_entry)
    label=None
    prob=-1
    for class_value,probability in probabilities.items():
        if label is None or probability > prob:
            prob = probability
            label = class_value
    return label

### getting predictions on complete validation_data

In [113]:
def get_predictions(mean_stdev_class, test_data):
    predictions = []
    for test_entry in test_data:
        result = predict(mean_stdev_class,train_data,test_entry)
        predictions.append(result)
    return predictions

### Getting accuracy

In [116]:
def get_accuracy(test_data,predictions):
    correct = 0
    for i in range(len(test_data)):
        if test_data[i][-1]==predictions[i]:
            correct += 1
    a,b,c,d=score(predictions,test_data[:,-1])
    print('Precision:',a[0])
    print('Recall:',b[0])
    print('F1:',c[0])
    return (correct/float(len(test_data))) * 100.0

### Training and testing the model with train_data and validation_data respectively

In [117]:
mean_stdev_by_class = get_mean_stdev_by_class(train_data)
predictions = get_predictions(mean_stdev_by_class,validation_data)
accuracy = get_accuracy(validation_data, predictions)
print('Accuracy',accuracy)

Precision: 0.8726823238566132
Recall: 0.9860335195530726
F1: 0.9259016393442624
Accuracy 87.44444444444444
